# Plan and Execute Pattern using Semantic Kernel

This notebook demonstrates the implementation of the Plan and Execute pattern using Semantic Kernel. This pattern improves agent performance by:

1. Breaking down complex tasks into manageable sub-tasks (Planning)
2. Executing each sub-task in sequence
3. Adapting to feedback during execution

## Architecture Overview

The Plan and Execute pattern involves:
- **Planner**: Responsible for generating a structured plan of sub-tasks
- **Executor**: Handles the execution of each sub-task
- **Memory**: Maintains context between steps
- **Tools**: Custom functions that can be called during execution

![Plan and Execute Pattern](../../../1_agentic-design-ptn/images/planning.png)

## Understanding the Plan and Execute Pattern

The Plan and Execute pattern we've demonstrated offers several advantages:

1. **Task Decomposition**: Complex tasks are broken down into simpler, manageable steps
2. **Tool Selection**: The planner automatically selects the appropriate tools for each step
3. **Adaptability**: If a step fails, the planner can adapt by trying alternative approaches
4. **Explainability**: The plan provides transparency into how the agent approaches problems

This pattern is particularly useful for tasks that require multiple steps or the use of various tools to complete.

## Practical Applications

The Plan and Execute pattern can be applied to various real-world scenarios:

- **Customer Support**: Helping customers troubleshoot complex issues step by step
- **Research Assistance**: Breaking down research questions into specific search queries and synthesis steps
- **Task Automation**: Creating workflows that combine multiple API calls and data transformations
- **Product Recommendations**: Gathering user preferences and matching them to suitable products

## Next Steps

To extend this pattern, consider:

1. Adding error handling to retry failed steps
2. Implementing dynamic replanning based on execution results
3. Incorporating user feedback between steps
4. Adding more specialized tools for specific domains

In [ ]:
# Setup and Dependencies
import os
import json
import requests
import asyncio
import httpx
from urllib.parse import urljoin
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.planners import SequentialPlanner
from semantic_kernel.planners.sequential_planner.sequential_planner_config import SequentialPlannerConfig
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from typing import List, Dict, Any
import nest_asyncio
from IPython.display import display, Markdown
import logging
import sys
sys.path.append(os.path.abspath('../../..'))  # Adjust the path as necessary
from utils.search_utils import web_search


# Apply nest_asyncio to allow nested event loops (required for Jupyter)
nest_asyncio.apply()

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Helper function to display markdown content
def display_md(text: str):
    display(Markdown(text))

In [3]:
# Load environment variables and configure Azure OpenAI
from dotenv import load_dotenv
load_dotenv()

# Get API keys from environment variables
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_KEY")
DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

# Check that environment variables are set
if not AZURE_OPENAI_ENDPOINT or not AZURE_OPENAI_API_KEY or not DEPLOYMENT_NAME:
    raise ValueError("Please set AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_KEY, and AZURE_OPENAI_DEPLOYMENT_NAME in .env file")
    
print(f"Azure OpenAI Endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"Deployment Name: {DEPLOYMENT_NAME}")

# Check if Google Search API keys are available
if not GOOGLE_API_KEY or not GOOGLE_CSE_ID:
    print("Warning: GOOGLE_API_KEY or GOOGLE_CSE_ID not set. Web search functionality will be limited to demo mode.")

ValueError: Please set AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_KEY, and AZURE_OPENAI_DEPLOYMENT_NAME in .env file

In [4]:


async def extract_text_from_url(url):
    """
    Extract text content from a webpage URL.
    
    Args:
        url: The URL to extract text from
        
    Returns:
        Extracted text content
    """
    # For demo purposes, we'll generate some simulated content
    # In a real implementation, you would use httpx to fetch and parse the content
    
    # Simple simulation of webpage content based on URL
    if "example.com" in url:
        product_type = ""
        for product in ["smartphone", "laptop", "headphones", "smartwatch", "tablet"]:
            if product in url:
                product_type = product
                break
        
        if product_type:
            return f"This is a webpage about {product_type}s. It contains detailed information about features, specifications, and user reviews."
        else:
            return "This is a generic technology webpage with product information and reviews."
    
    return "Could not extract content from the provided URL."

async def add_context_async(urls):
    """
    Fetch and extract content from multiple URLs asynchronously.
    
    Args:
        urls: List of URLs to fetch content from
        
    Returns:
        List of extracted text content
    """
    tasks = [extract_text_from_url(url) for url in urls]
    results = await asyncio.gather(*tasks)
    return results

In [5]:
# Initialize Semantic Kernel with Azure OpenAI
kernel = sk.Kernel()

# Add Azure OpenAI service
kernel.add_service(
    AzureChatCompletion(
        service_id="azure_chat_completion",
        deployment_name=DEPLOYMENT_NAME,
        endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY
    )
)

print("Semantic Kernel initialized with Azure OpenAI service")

Semantic Kernel initialized with Azure OpenAI service


## Creating Native Functions (Tools)

First, we'll create several native functions that our planner can use during execution. These functions represent the tools that our agent can use to complete tasks.

In [6]:
# Create a plugin with native functions that can be used by the planner
@kernel_function(
    description="Searches for information about a product.",
    name="search_product"
)
def search_product(query: str) -> str:
    """
    Searches for information about a specified product using web search.
    
    Args:
        query: The product to search for.
        
    Returns:
        Information about the product.
    """
    logger.info(f"Searching for product: {query}")
    
    # First try web search
    results = web_search(query, num=3)
    
    if results:
        # Process search results
        product_info = f"Found information about {query} from web search:\n\n"
        
        for i, result in enumerate(results):
            title = result.get("title", "Untitled")
            link = result.get("link", "")
            snippet = result.get("snippet", result.get("text_content", ""))
            
            product_info += f"{i+1}. {title}\n"
            product_info += f"   URL: {link}\n"
            if snippet:
                product_info += f"   Summary: {snippet}\n"
            product_info += "\n"
            
        return product_info
    
    return f"Could not find specific information about '{query}'. Available products in our local database are: {', '.join(static_product_database.keys())}."

@kernel_function(
    description="Compares features between two products.",
    name="compare_products"
)
def compare_products(product1: str, product2: str) -> str:
    """
    Compares features between two products using web search.
    
    Args:
        product1: First product to compare.
        product2: Second product to compare.
        
    Returns:
        Comparison between the products.
    """
    logger.info(f"Comparing products: {product1} vs {product2}")
    
    # Search for each product
    product1_results = web_search(product1, num=2)
    product2_results = web_search(product2, num=2)
    
    if product1_results and product2_results:
        comparison = f"Comparison between {product1} and {product2} based on web search:\n\n"
        
        # First product info
        comparison += f"## {product1.capitalize()} Information:\n"
        for i, result in enumerate(product1_results[:2]):
            title = result.get("title", "Untitled")
            snippet = result.get("snippet", result.get("text_content", "No description available"))
            comparison += f"{i+1}. {title}\n   {snippet}\n\n"
        
        # Second product info
        comparison += f"## {product2.capitalize()} Information:\n"
        for i, result in enumerate(product2_results[:2]):
            title = result.get("title", "Untitled")
            snippet = result.get("snippet", result.get("text_content", "No description available"))
            comparison += f"{i+1}. {title}\n   {snippet}\n\n"
        
        # Add comparison prompt
        comparison += f"Based on the information above, here are the key differences between {product1} and {product2}:"
        
        return comparison
    
    return f"No comparison data available between '{product1}' and '{product2}'. Try with more specific product types."

@kernel_function(
    description="Provides recommendations based on user preferences.",
    name="recommend_product"
)
def recommend_product(preferences: str) -> str:
    """
    Recommends products based on user preferences using web search.
    
    Args:
        preferences: User preferences for product recommendations.
        
    Returns:
        Product recommendations.
    """
    logger.info(f"Generating recommendations based on preferences: {preferences}")
    
    # Create a search query based on user preferences
    search_query = f"best products for {preferences}"
    
    # Perform web search
    results = web_search(search_query, num=3)
    
    if results:
        recommendations = f"Based on your preferences for '{preferences}', here are some recommended products:\n\n"
        
        for i, result in enumerate(results):
            title = result.get("title", "Untitled")
            snippet = result.get("snippet", result.get("text_content", "No description available"))
            
            recommendations += f"{i+1}. {title}\n"
            recommendations += f"   {snippet}\n\n"
            
        return recommendations
    
    return "Based on your preferences (from local database), I might need more specific information to make a tailored recommendation."

# Register functions with the kernel
# Register each function as a plugin
kernel.add_function(plugin_name="product_plugin", function=search_product)
kernel.add_function(plugin_name="product_plugin", function=compare_products)
kernel.add_function(plugin_name="product_plugin", function=recommend_product)

print("Native functions registered as tools")

Native functions registered as tools


## Creating Semantic Functions

Next, we'll create semantic functions that use natural language to perform tasks. These will be combined with our native functions in the planner.

In [7]:
# Create a semantic function for summarizing product information
summarize_prompt = """
You are a helpful product specialist.
Summarize the following product information in a concise and helpful manner.
Focus on the key features and benefits that would be most relevant to customers.

Product Information:
{{$input}}

Summary:
"""

summarize_function = kernel.add_function(
    function_name="summarize_product_info",
    plugin_name="product_plugin",
    prompt=summarize_prompt,
    description="Summarizes product information in a concise and helpful manner."
)

# Create a semantic function for generating product comparisons
compare_prompt = """
You are a helpful product specialist.
Create a detailed comparison between the products based on the information provided.
Include pros and cons for each product and mention which types of users would prefer each option.

Product Information:
{{$input}}

Comparison:
"""

compare_function = kernel.add_function(
    function_name="generate_detailed_comparison",
    plugin_name="product_plugin",
    prompt=compare_prompt,
    description="Generates a detailed comparison between products."
)

print("Semantic functions created")

Semantic functions created


## Setting Up the Planner

Now we'll configure the sequential planner, which will break down complex tasks into a sequence of steps using our available functions.

In [8]:
# Configure the sequential planner
planner_config = SequentialPlannerConfig(
    relevancy_threshold=0.7,
    max_tokens=1000,
    included_functions=["search_product", "compare_products", "recommend_product", "summarize_product_info", "generate_detailed_comparison"],
)

# Create the planner with the config as a named parameter
planner = SequentialPlanner(service_id="azure_chat_completion", kernel=kernel, config=planner_config)

print("Sequential planner configured")

Sequential planner configured


## Executing a Plan

Let's put our plan and execute pattern to work with a sample scenario. We'll ask the agent to help a customer find the right product based on their needs.

In [9]:
async def execute_plan_and_display(goal: str):
    
    
    
    """Execute a plan based on the specified goal and display the results."""
    print(f"🎯 Goal: {goal}")
    print("\n⚙️ Generating plan...\n")
    
    # Create and display the plan
    plan = await planner.create_plan(goal)
    plan_steps = "\n".join([f"Step {i+1}: {step.description}" for i, step in enumerate(plan.steps)])
    display_md(f"### Generated Plan\n{plan_steps}")
    
    print("\n🚀 Executing plan...\n")
    
    # Execute the plan
    result = await plan.invoke(kernel=kernel)
    print("\n✅ Plan execution completed\n")
    display_md(f"### Result\n{result}")
    
    return plan, result

# Example goal
goal = "I need a recommendation for a product with good battery life for a college student."

# Execute the plan
await execute_plan_and_display(goal)

2025-05-19 08:44:08,708 - semantic_kernel.functions.kernel_function - INFO - Function SequentialPlanner_Excluded-SequentialPlanner_Excluded invoking.
2025-05-19 08:44:08,712 - semantic_kernel.contents.chat_history - INFO - Could not parse prompt Create an XML plan step by step, to satisfy the goal given, with the available functions.

[AVAILABLE FUNCTIONS]

product_plugin-compare_products:
  description: Compares features between two products.
  inputs:
    - product1: None
  - product2: None

product_plugin-generate_detailed_comparison:
  description: Generates a detailed comparison between products.
  inputs:
    - input: 

product_plugin-recommend_product:
  description: Provides recommendations based on user preferences.
  inputs:
    - preferences: None

product_plugin-search_product:
  description: Searches for information about a product.
  inputs:
    - query: None

product_plugin-summarize_product_info:
  description: Summarizes product information in a concise and helpful man

🎯 Goal: I need a recommendation for a product with good battery life for a college student.

⚙️ Generating plan...



2025-05-19 08:44:09,525 - httpx - INFO - HTTP Request: POST https://aoai-services1.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"
2025-05-19 08:44:09,529 - semantic_kernel.connectors.ai.open_ai.services.open_ai_handler - INFO - OpenAI usage: CompletionUsage(completion_tokens=59, prompt_tokens=908, total_tokens=967, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
2025-05-19 08:44:09,530 - semantic_kernel.functions.kernel_function - INFO - Function SequentialPlanner_Excluded-SequentialPlanner_Excluded succeeded.
2025-05-19 08:44:09,531 - semantic_kernel.functions.kernel_function - INFO - Function completed. Duration: 0.821238s


### Generated Plan
Step 1: Provides recommendations based on user preferences.

2025-05-19 08:44:09,536 - semantic_kernel.planners.plan - INFO - Invoking next step: recommend_product with arguments: {}
2025-05-19 08:44:09,537 - semantic_kernel.functions.kernel_function - INFO - Function product_plugin-recommend_product invoking.
2025-05-19 08:44:09,537 - __main__ - INFO - Generating recommendations based on preferences: good battery life for a college student
2025-05-19 08:44:09,538 - utils.search_utils - INFO - Using Bing Grounding search for query: best products for good battery life for a college student
2025-05-19 08:44:09,538 - utils.search_utils - WARNING - Bing Grounding connection string not set. Using fallback to Google search.



🚀 Executing plan...



2025-05-19 08:44:09,745 - semantic_kernel.functions.kernel_function - INFO - Function product_plugin-recommend_product succeeded.
2025-05-19 08:44:09,746 - semantic_kernel.functions.kernel_function - INFO - Function completed. Duration: 0.208353s
2025-05-19 08:44:09,746 - semantic_kernel.planners.plan - INFO - updated arguments: {'input': 'Based on your preferences for \'good battery life for a college student\', here are some recommended products:\n\n1. Best Student Laptop | Find The Best Galaxy Book For You ...\n   Choosing a laptop with long battery life will stop any interruptions to your ... What Is The Best Samsung Laptop For Students? Footer Navigation. Product &\xa0...\n\n2. Best Tablet For Students: Study Smarter With Samsung Galaxy Tab\n   Mar 10, 2022 ... ... student life a whole lot easier. And with ... If you\'re looking for a Galaxy Tab with a long lasting battery look no further than the three\xa0...\n\n3. Laptop or Tablet: Which One Should You Buy? | Samsung UK\n   To h


✅ Plan execution completed



### Result
Based on your preferences for 'good battery life for a college student', here are some recommended products:

1. Best Student Laptop | Find The Best Galaxy Book For You ...
   Choosing a laptop with long battery life will stop any interruptions to your ... What Is The Best Samsung Laptop For Students? Footer Navigation. Product & ...

2. Best Tablet For Students: Study Smarter With Samsung Galaxy Tab
   Mar 10, 2022 ... ... student life a whole lot easier. And with ... If you're looking for a Galaxy Tab with a long lasting battery look no further than the three ...

3. Laptop or Tablet: Which One Should You Buy? | Samsung UK
   To help you with your decision-making, check out our “Best Tablet For Students” and "Best Laptop For Students" guides. ... Long lasting battery life. Physical ...



(<semantic_kernel.planners.plan.Plan at 0x7ff7dcce5650>,
 FunctionResult(function=KernelFunctionMetadata(name='plan_kLDVFMZWMbNvCjgk', plugin_name='Plan', description='I need a recommendation for a product with good battery life for a college student.', parameters=[], is_prompt=False, is_asynchronous=True, return_parameter=None, additional_properties=None), value='Based on your preferences for \'good battery life for a college student\', here are some recommended products:\n\n1. Best Student Laptop | Find The Best Galaxy Book For You ...\n   Choosing a laptop with long battery life will stop any interruptions to your ... What Is The Best Samsung Laptop For Students? Footer Navigation. Product &\xa0...\n\n2. Best Tablet For Students: Study Smarter With Samsung Galaxy Tab\n   Mar 10, 2022 ... ... student life a whole lot easier. And with ... If you\'re looking for a Galaxy Tab with a long lasting battery look no further than the three\xa0...\n\n3. Laptop or Tablet: Which One Should You B

## Try Different Goals

Let's try different types of user requests to see how our planner adapts.

In [9]:
# Let's try a comparison request
comparison_goal = "Compare the features of a samsung laptop and a samsung tablet for someone who needs to do both schoolwork and watch movies."

await execute_plan_and_display(comparison_goal)

2025-05-19 08:39:27,118 - semantic_kernel.functions.kernel_function - INFO - Function SequentialPlanner_Excluded-SequentialPlanner_Excluded invoking.
2025-05-19 08:39:27,119 - semantic_kernel.contents.chat_history - INFO - Could not parse prompt Create an XML plan step by step, to satisfy the goal given, with the available functions.

[AVAILABLE FUNCTIONS]

product_plugin-compare_products:
  description: Compares features between two products.
  inputs:
    - product1: None
  - product2: None

product_plugin-generate_detailed_comparison:
  description: Generates a detailed comparison between products.
  inputs:
    - input: 

product_plugin-recommend_product:
  description: Provides recommendations based on user preferences.
  inputs:
    - preferences: None

product_plugin-search_product:
  description: Searches for information about a product.
  inputs:
    - query: None

product_plugin-summarize_product_info:
  description: Summarizes product information in a concise and helpful man

🎯 Goal: Compare the features of a samsung laptop and a samsung tablet for someone who needs to do both schoolwork and watch movies.

⚙️ Generating plan...



2025-05-19 08:39:29,121 - httpx - INFO - HTTP Request: POST https://aoai-services1.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"
2025-05-19 08:39:29,123 - semantic_kernel.connectors.ai.open_ai.services.open_ai_handler - INFO - OpenAI usage: CompletionUsage(completion_tokens=170, prompt_tokens=917, total_tokens=1087, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
2025-05-19 08:39:29,124 - semantic_kernel.functions.kernel_function - INFO - Function SequentialPlanner_Excluded-SequentialPlanner_Excluded succeeded.
2025-05-19 08:39:29,125 - semantic_kernel.functions.kernel_function - INFO - Function completed. Duration: 2.006078s
2025-05-19 08:39:29,123 - semantic_kernel.connectors.ai.open_ai.services.open_ai_handler - INFO - OpenAI usage: CompletionU

### Generated Plan
Step 1: Searches for information about a product.
Step 2: Searches for information about a product.
Step 3: Generates a detailed comparison between products.
Step 4: Summarizes product information in a concise and helpful manner.

2025-05-19 08:39:29,127 - semantic_kernel.planners.plan - INFO - Invoking next step: search_product with arguments: {}
2025-05-19 08:39:29,128 - semantic_kernel.functions.kernel_function - INFO - Function product_plugin-search_product invoking.
2025-05-19 08:39:29,129 - __main__ - INFO - Searching for product: Samsung laptop
2025-05-19 08:39:29,129 - utils.search_utils - INFO - Using Bing Grounding search for query: Samsung laptop
2025-05-19 08:39:29,130 - azure.identity._credentials.environment - INFO - No environment configuration found.
2025-05-19 08:39:29,130 - azure.identity._credentials.managed_identity - INFO - ManagedIdentityCredential will use Azure ML managed identity
2025-05-19 08:39:29,131 - utils.search_utils - INFO - Using existing agent ID: asst_1BNu5p4Wv52Cload5HkW3ZBa
2025-05-19 08:39:29,133 - azure.core.pipeline.policies.http_logging_policy - INFO - Request URL: 'http://127.0.0.1:46808/MSI/auth?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request head


🚀 Executing plan...



2025-05-19 08:39:29,407 - azure.core.pipeline.policies.http_logging_policy - INFO - Response status: 200
Response headers:
    'Date': 'Mon, 19 May 2025 08:39:29 GMT'
    'Content-Type': 'application/json'
    'Transfer-Encoding': 'chunked'
    'Connection': 'keep-alive'
    'Vary': 'REDACTED'
    'Request-Context': 'REDACTED'
    'x-ms-response-type': 'REDACTED'
    'x-ms-middleware-request-id': 'REDACTED'
    'openai-version': 'REDACTED'
    'openai-organization': 'REDACTED'
    'X-Request-ID': 'REDACTED'
    'openai-processing-ms': 'REDACTED'
    'Strict-Transport-Security': 'REDACTED'
    'apim-request-id': 'REDACTED'
    'X-Content-Type-Options': 'REDACTED'
    'x-ms-region': 'REDACTED'
    'azureml-served-by-cluster': 'REDACTED'
    'x-request-time': 'REDACTED'
    'Content-Encoding': 'REDACTED'
2025-05-19 08:39:29,409 - azure.core.pipeline.policies.http_logging_policy - INFO - Request URL: 'https://swedencentral.api.azureml.ms/agents/v1.0/subscriptions/3d4d3dd0-79d4-40cf-a94e-b4


✅ Plan execution completed



### Result
### Samsung Tablet Comparison Summary

#### Samsung Galaxy Tab S8
- **Display:** 11-inch TFT LCD, 2560 x 1600 resolution
- **Processor:** Qualcomm Snapdragon 8 Gen 1
- **RAM:** Options of 8GB / 12GB
- **Storage:** 128GB / 256GB (Expandable via MicroSD)
- **Battery Life:** Up to 15 hours
- **Software:** Android 12 with One UI 4.1
- **Weight:** Approx. 503 grams
- **Stylus Support:** S Pen included

**Key Benefits:**
- High-resolution display ideal for media and graphics
- Powerful processor for seamless multitasking and gaming
- Long battery life suitable for extended usage
- Lightweight for on-the-go convenience
- Expandable storage for managing files easily

**Ideal For:**
- Professionals needing a portable workstation
- Students for note-taking and studying
- Gamers seeking high performance

---

#### Samsung Galaxy Tab A7
- **Display:** 10.4-inch TFT LCD, 2000 x 1200 resolution
- **Processor:** Qualcomm Snapdragon 662
- **RAM:** Options of 3GB / 4GB
- **Storage:** 32GB / 64GB (Expandable via MicroSD)
- **Battery Life:** Up to 13 hours
- **Software:** Android 10 (upgradable)
- **Weight:** Approx. 476 grams
- **Stylus Support:** No S Pen included

**Key Benefits:**
- Affordable price point for wider accessibility
- Good performance for everyday tasks like browsing and streaming
- Sleek and lightweight design for portability
- Decent battery life for casual use

**Ideal For:**
- Casual users for browsing and media
- Families or kids due to its durability
- Educational purposes with light apps and web access

### Conclusion
The **Samsung Galaxy Tab S8** is best for those who require performance and productivity, ideal for professionals and gamers. The **Samsung Galaxy Tab A7** is perfect for budget-conscious users and families looking for a reliable tablet for everyday tasks.

(<semantic_kernel.planners.plan.Plan at 0x7f20971dd110>,
 FunctionResult(function=KernelFunctionMetadata(name='plan_mevWJfOHplHTikUr', plugin_name='Plan', description='Compare the features of a samsung laptop and a samsung tablet for someone who needs to do both schoolwork and watch movies.', parameters=[], is_prompt=False, is_asynchronous=True, return_parameter=None, additional_properties=None), value='### Samsung Tablet Comparison Summary\n\n#### Samsung Galaxy Tab S8\n- **Display:** 11-inch TFT LCD, 2560 x 1600 resolution\n- **Processor:** Qualcomm Snapdragon 8 Gen 1\n- **RAM:** Options of 8GB / 12GB\n- **Storage:** 128GB / 256GB (Expandable via MicroSD)\n- **Battery Life:** Up to 15 hours\n- **Software:** Android 12 with One UI 4.1\n- **Weight:** Approx. 503 grams\n- **Stylus Support:** S Pen included\n\n**Key Benefits:**\n- High-resolution display ideal for media and graphics\n- Powerful processor for seamless multitasking and gaming\n- Long battery life suitable for extended usag

In [10]:
# Let's try a specific product search request
search_goal = "I want to know the camera quality and storage options for the smartphone."

await execute_plan_and_display(search_goal)

2025-05-19 08:44:28,625 - semantic_kernel.functions.kernel_function - INFO - Function SequentialPlanner_Excluded-SequentialPlanner_Excluded invoking.
2025-05-19 08:44:28,627 - semantic_kernel.contents.chat_history - INFO - Could not parse prompt Create an XML plan step by step, to satisfy the goal given, with the available functions.

[AVAILABLE FUNCTIONS]

product_plugin-compare_products:
  description: Compares features between two products.
  inputs:
    - product1: None
  - product2: None

product_plugin-generate_detailed_comparison:
  description: Generates a detailed comparison between products.
  inputs:
    - input: 

product_plugin-recommend_product:
  description: Provides recommendations based on user preferences.
  inputs:
    - preferences: None

product_plugin-search_product:
  description: Searches for information about a product.
  inputs:
    - query: None

product_plugin-summarize_product_info:
  description: Summarizes product information in a concise and helpful man

🎯 Goal: I want to know the camera quality and storage options for the smartphone.

⚙️ Generating plan...



2025-05-19 08:44:30,186 - httpx - INFO - HTTP Request: POST https://aoai-services1.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"
2025-05-19 08:44:30,188 - semantic_kernel.connectors.ai.open_ai.services.open_ai_handler - INFO - OpenAI usage: CompletionUsage(completion_tokens=143, prompt_tokens=906, total_tokens=1049, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
2025-05-19 08:44:30,189 - semantic_kernel.functions.kernel_function - INFO - Function SequentialPlanner_Excluded-SequentialPlanner_Excluded succeeded.
2025-05-19 08:44:30,190 - semantic_kernel.functions.kernel_function - INFO - Function completed. Duration: 1.563757s


### Generated Plan
Step 1: Searches for information about a product.
Step 2: Summarizes product information in a concise and helpful manner.
Step 3: Generates a detailed comparison between products.

2025-05-19 08:44:30,193 - semantic_kernel.planners.plan - INFO - Invoking next step: search_product with arguments: {}
2025-05-19 08:44:30,194 - semantic_kernel.functions.kernel_function - INFO - Function product_plugin-search_product invoking.
2025-05-19 08:44:30,195 - __main__ - INFO - Searching for product: smartphone
2025-05-19 08:44:30,195 - utils.search_utils - INFO - Using Bing Grounding search for query: smartphone
2025-05-19 08:44:30,196 - utils.search_utils - WARNING - Bing Grounding connection string not set. Using fallback to Google search.



🚀 Executing plan...



2025-05-19 08:44:30,533 - semantic_kernel.functions.kernel_function - INFO - Function product_plugin-search_product succeeded.
2025-05-19 08:44:30,533 - semantic_kernel.functions.kernel_function - INFO - Function completed. Duration: 0.338833s
2025-05-19 08:44:30,534 - semantic_kernel.planners.plan - INFO - updated arguments: {'input': 'Found information about smartphone from web search:\n\n1. Explore the Latest Galaxy Smartphones | Samsung US\n   URL: https://www.samsung.com/us/smartphones/\n   Summary: Discover Galaxy smartphones. A phone as smart as you are. Switch to Galaxy S25 Series and save up to $630 with trade-in.\n\n2. Galaxy Cell Phones | Shop All Smartphones | Samsung US\n   URL: https://www.samsung.com/us/mobile/phones/all-phones/\n   Summary: Buy in bulk for your business and get exclusive volume discounts, 0% Samsung Business Financing, bulk trade-in credit and free shipping, plus tax-free shopping.\n\n3. Samsung US | Mobile | TV | Home Electronics | Home Appliances ...\


✅ Plan execution completed



### Result
### Product Comparison: Samsung Galaxy S25 Smartphones

#### Overview
The Samsung Galaxy S25 series represents the latest offering from Samsung, integrating cutting-edge technology with user-friendly design. The series is designed for both individual consumers and businesses, with features and benefits that cater to diverse user needs.

---

### Key Features

1. **Advanced Technology**: The Galaxy S25 series is equipped with the latest smartphone innovations, which can include advanced camera systems, high-refresh-rate displays, and powerful processors suited for multitasking.
   
2. **Trade-In Savings**: Up to $630 in savings for customers who participate in eligible trade-in programs. This makes upgrading easier and more economical.

3. **Business-Focused Bulk Purchasing**: Samsung's bulk purchasing program provides businesses with exclusive discounts, 0% financing options, bulk trade-in credits, and free shipping, enhancing the cost-effectiveness for companies needing multiple devices.

4. **Wide Range of Products**: In addition to smartphones, Samsung offers an impressive array of electronic products, allowing consumers to build a cohesive tech ecosystem.

---

### **Pros & Cons**

#### Samsung Galaxy S25 Series

**Pros:**
- **Cutting-Edge Technology**: Equipped with high performance features ideal for tech enthusiasts and power users.
- **Flexible Savings Programs**: The trade-in program significantly lowers the cost of upgrading, making the latest technology more accessible.
- **Business Adaptability**: The bulk purchasing options make it ideal for businesses needing to equip multiple employees with devices.
- **Ecosystem Integration**: Ability to seamlessly integrate with a wide range of Samsung products enhances user experience.

**Cons:**
- **Premium Pricing**: The initial cost of the Galaxy S25 may be high for some consumers, especially if they do not have trade-ins.
- **Complexity of Features**: Advanced technology may overwhelm less tech-savvy users or those looking for a simple smartphone experience.
- **Availability Limitations**: Certain promotional offers may be limited by region or product availability, which could affect some users.

---

### **Ideal User Profiles**

1. **Tech Enthusiasts**: Users who are passionate about technology and want the latest features will appreciate the advanced capabilities of the Galaxy S25 series.

2. **Budget-Conscious Consumers**: Those looking to upgrade their smartphone without incurring high costs can benefit from the substantial trade-in savings.

3. **Business Professionals**: Organizations that require multiple devices can take advantage of the bulk purchasing program, making it a cost-effective option for equipping a workforce.

4. **Brand Loyalists**: Users who already own Samsung products will benefit from the synergy and integration of the Galaxy S25 with other Samsung devices (e.g., wearables, home appliances).

5. **Non-Tech Savvy Users**: Individuals seeking a straightforward smartphone experience may find the advanced features excessive, leading them to prefer simpler devices.

---

### Conclusion

The Samsung Galaxy S25 series stands out for its blend of advanced technology, significant savings through trade-ins, and business-oriented purchasing options. However, the premium pricing and complexity of features may not appeal to every user. This smartphone line is ideally suited for tech enthusiasts, businesses in need of bulk devices, and existing Samsung users looking to expand their ecosystem. Users who prioritize simplicity might want to consider alternatives that offer a less complex experience.

(<semantic_kernel.planners.plan.Plan at 0x7ff7dcd1ab90>,
 FunctionResult(function=KernelFunctionMetadata(name='plan_jZZiUxDJTivJSfJW', plugin_name='Plan', description='I want to know the camera quality and storage options for the smartphone.', parameters=[], is_prompt=False, is_asynchronous=True, return_parameter=None, additional_properties=None), value="### Product Comparison: Samsung Galaxy S25 Smartphones\n\n#### Overview\nThe Samsung Galaxy S25 series represents the latest offering from Samsung, integrating cutting-edge technology with user-friendly design. The series is designed for both individual consumers and businesses, with features and benefits that cater to diverse user needs.\n\n---\n\n### Key Features\n\n1. **Advanced Technology**: The Galaxy S25 series is equipped with the latest smartphone innovations, which can include advanced camera systems, high-refresh-rate displays, and powerful processors suited for multitasking.\n   \n2. **Trade-In Savings**: Up to $630 in savin